Note: Recommended to run all cells from top to avoid errors relating to context not being open

In [1]:
from pylogic.theories.numbers import Naturals, Rationals, Integers
from pylogic.assumptions_context import AssumptionsContext, conclude
from pylogic.expressions.expr import sqrt
from pylogic.proposition.not_ import Not

root_2  = sqrt(2)
root_2

Pow(Constant(2, deps=()), Constant(1/2, deps=()))

In [2]:
ctx = AssumptionsContext()
ctx.open()

# assume that root_2 is rational
rt2_rational = root_2.is_in(Rationals).assume()
rt2_rational


IsContainedIn(2^1/2, Rationals)

In [3]:
display(Rationals.theorems.ratio_of_integers_lowest_terms)

# this means that root_2 = p/q for some integers p and q, q != 0
# and gcd(p, q) = 1  , call this Equation 1
rt2_ratio = Rationals.theorems.ratio_of_integers_lowest_terms(root_2)
display(rt2_ratio)

ForallInSet(Variable(r, deps=()), Set_Rationals, ExistsInSet(Variable(p, deps=()), Set_Integers, ExistsInSet(Variable(q, deps=()), Set_Integers, And(Not(Equals(q, 0)), Equals(r, p * q^-1), Equals(gcd(p, q), 1)))))

ExistsInSet(Variable(p, deps=()), Set_Integers, ExistsInSet(Variable(q, deps=()), Set_Integers, And(Not(Equals(q, 0)), Equals(2^1/2, p * q^-1), Equals(gcd(p, q), 1))))

In [4]:
# example of a nested structure to destructure

# a, (b, (c, d)) = [1, [2, [3, 4]]]
# print(repr(rt2_ratio))
print(rt2_ratio.as_text())

exists p:
    IsContainedIn(p, Integers)
  and
    exists q:
        IsContainedIn(q, Integers)
      and
        it is false that
          Equals(q, 0)
      and
        Equals(2^1/2, p * q^-1)
      and
        Equals(gcd(p, q), 1)



In [5]:
# p, p_integer_and_q_etc = rt2_ratio
# p_integer, q_q_integer_and_etc = p_integer_and_q_etc
# q, q_integer_and_etc = q_q_integer_and_etc
# q_integer, q_nonzero, rt2_eq_p_over_q, gcd_p_q_eq_1 = q_integer_and_etc


p, (_, (q, (_, _, rt2_eq_p_over_q, gcd_p_q_eq_1))) = rt2_ratio

display(rt2_eq_p_over_q)
display(gcd_p_q_eq_1)


Equals(2^1/2, p * q^-1)

Equals(gcd(p, q), 1)

In [6]:
# 2 is prime
two_prime = Naturals.prime(2).by_inspection()
display(two_prime)

display(two_prime.definition)

Prime(2)

And(2 > 1, ForallInSet(Variable(a, deps=()), Set_Naturals, ForallInSet(Variable(b, deps=()), Set_Naturals, Implies(Equals(2, a * b), ExOr(Equals(a, 1), Equals(b, 1))))))

In [7]:
# root_2 = p/q
# =>   2 = p^2/q^2
# =>   2q^2 = p^2
# =>   p^2 = 2q^2, call this Equation 2
p2_eq_2q2 = ((rt2_eq_p_over_q ** 2) * q**2).evaluate().symmetric()

# (root_2)^2 = (p/q)^2
# =>   (root_2)^2 * q^2 = (p/q)^2 * q^2
# evaluate => 2q^2 = p^2


In [8]:
print("Product division theorem")
display(Integers.theorems.division_theorems.product)
# by the above theorem, 2 | p^2
two_div_p2, _ = Integers.theorems.division_theorems.product(p**2, 2, q**2)(p2_eq_2q2)
display(two_div_p2)

# display(Integers.theorems.division_theorems.product(p**2, 2, q**2))


Product division theorem


ForallInSet(Variable(a, deps=()), Set_Integers, ForallInSet(Variable(b, deps=()), Set_Integers, ForallInSet(Variable(c, deps=()), Set_Integers, Implies(Equals(a, b * c), And(Divides(b, a, Integers), Divides(c, a, Integers))))))

Divides(2, p^2, Integers)

In [9]:
print("Prime divides power theorem")
display(Integers.theorems.division_theorems.prime.prime_divides_power)
# by the above theorem, 2 | p
two_div_p = Integers.theorems.division_theorems.prime.prime_divides_power(2, p, 2)(two_prime, two_div_p2)



Prime divides power theorem


ForallInSet(Variable(p, deps=()), Set_Naturals, ForallInSet(Variable(a, deps=()), Set_Integers, ForallInSet(Variable(k, deps=()), Set_Naturals, Implies(And(Prime(p), Divides(p, a^k, Integers)), Divides(p, a, Integers)))))

In [10]:
# just the definition of divides, rename the variable as k
display(two_div_p.definition)
display(two_div_p.definition.rename_variable('k'))
k, (_, p_eq_2k) = two_div_p.definition.rename_variable('k') # p = 2k

display(p_eq_2k)

ExistsInSet(Variable(q, deps=()), Set_Integers, Equals(p, 2 * q))

ExistsInSet(Variable(k, deps=()), Set_Integers, Equals(p, 2 * k))

Equals(p, 2 * k)

In [11]:
# substitute Equation 2 (p^2 = 2q^2) right-hand-side into p^2 = (2k)^2
# => 2q^2 = (2k)^2
# => q^2 = 2k^2
q2_eq_2k2 = (p_eq_2k ** 2).substitute("right", p2_eq_2q2) / 2
q2_eq_2k2 = q2_eq_2k2.evaluate()



In [12]:
# again, product division theorem, 2 | q^2
two_div_q2, _ = Integers.theorems.division_theorems.product(q**2, 2, k**2)(q2_eq_2k2)
# and prime divides power theorem, 2 | q
two_div_q = Integers.theorems.division_theorems.prime.prime_divides_power(2, q, 2)(two_prime, two_div_q2)



In [13]:
display(Integers.theorems.division_theorems.gcd)

# now we have 2 | p and 2 | q
# => 2 | gcd(p, q)
two_div_gcd_p_q = Integers.theorems.division_theorems.gcd(2, p, q)(two_div_p, two_div_q)
display(two_div_gcd_p_q)



ForallInSet(Variable(a, deps=()), Set_Integers, ForallInSet(Variable(b, deps=()), Set_Integers, ForallInSet(Variable(c, deps=()), Set_Integers, Implies(And(Divides(a, b, Integers), Divides(a, c, Integers)), Divides(a, gcd(b, c), Integers)))))

Divides(2, gcd(p, q), Integers)

In [14]:
# but we know that gcd(p, q) = 1 (Equation 1), so 2 | 1
two_div_1 = two_div_gcd_p_q.substitute("right", gcd_p_q_eq_1)
display(two_div_1)


Divides(2, 1, Integers)

In [15]:
# 2 clearly does not divide 1, so contradiction
two_not_div_1 = Not(Integers.divides(2, 1)).by_inspection()
display(two_not_div_1)

Not(Divides(2, 1, Integers))

In [16]:
contra = two_div_1.contradicts(two_not_div_1)

conclude(contra)
display(contra)


ctx.close()


Proposition(contradiction)

In [17]:
ctx.get_proven()[0]

Not(IsContainedIn(2^1/2, Rationals))